<a href="https://colab.research.google.com/github/mohamedchahed/Omdena/blob/main/shoplifting_video_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fvcore

In [2]:
pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=4e76f32c779c356fd6c879d24adb7a88dd5032900e9b28b2909895679332b233
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [3]:
pip install timm

In [4]:
pip install iopath

In [5]:
pip install av

In [6]:
pip install pytorch_lightning

In [7]:
import numpy as np
from pytorch_lightning import seed_everything, LightningModule, Trainer
import torch.nn as nn
import torch
from torch.utils.data.dataloader import DataLoader
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint,LearningRateMonitor
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau,CosineAnnealingWarmRestarts,OneCycleLR,CosineAnnealingLR
import torchvision
import pandas as pd
import numpy as np
from glob import glob
from PIL import Image
import cv2
import os
from torch.utils.data import DataLoader, Dataset,ConcatDataset,default_collate
from sklearn.model_selection import KFold,GroupShuffleSplit,GroupKFold,LeaveOneGroupOut
from torchmetrics import MeanAbsoluteError
from sklearn.utils import shuffle
import shutil
from sklearn.model_selection import train_test_split
from torchaudio import transforms as TA
from sklearn.metrics import classification_report
import torchmetrics

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import sys
sys.path.append("/content/drive/MyDrive/Vision/pytorchvideo")

In [10]:
import pandas as pd
import os

normal_pth = "/content/drive/MyDrive/Vision/Shoplifting Dataset (2022) - CV Laboratory MNNIT Allahabad/Dataset/Normal"
anom_pth = "/content/drive/MyDrive/Vision/Shoplifting Dataset (2022) - CV Laboratory MNNIT Allahabad/Dataset/Shoplifting"

normal_files = [normal_pth + "/" + i for i in os.listdir(normal_pth)]
anom_files = [anom_pth + "/" + i for i in os.listdir(anom_pth)]

file_paths = normal_files + anom_files

df = pd.DataFrame({'path': file_paths})
df["path"][0]

'/content/drive/MyDrive/Vision/Shoplifting Dataset (2022) - CV Laboratory MNNIT Allahabad/Dataset/Normal/Normal (1).mp4'

In [11]:
df["label"] = [0]*len(normal_files) + [1]*len(anom_files)
df

,path,label
0,/content/drive/MyDrive/Vision/Shoplifting Data...,0
1,/content/drive/MyDrive/Vision/Shoplifting Data...,0
2,/content/drive/MyDrive/Vision/Shoplifting Data...,0
3,/content/drive/MyDrive/Vision/Shoplifting Data...,0
4,/content/drive/MyDrive/Vision/Shoplifting Data...,0
...,...,...
177,/content/drive/MyDrive/Vision/Shoplifting Data...,1
178,/content/drive/MyDrive/Vision/Shoplifting Data...,1
179,/content/drive/MyDrive/Vision/Shoplifting Data...,1
180,/content/drive/MyDrive/Vision/Shoplifting Data...,1


In [12]:
from sklearn.model_selection import train_test_split
train_df,val_df=train_test_split(df,test_size=0.25)

In [13]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [21]:
#tuneable params
num_video_samples=20
video_duration=2
model_name='efficient_x3d_xs'
batch_size=8
scheduler='cosine'
clipmode='random'
img_size=224

In [22]:
video_transform = Compose(
            [
            ApplyTransformToKey(
              key="video",
              transform=Compose(
                  [
                    UniformTemporalSubsample(num_video_samples),
                    Lambda(lambda x: x / 255.0),
                    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                      #Determines the shorter spatial dim of the video (i.e. width or height) and scales it to the given size
                    RandomShortSideScale(min_size=img_size+16, max_size=img_size+32),
                    CenterCropVideo(img_size),
                    RandomHorizontalFlip(p=0.5),
                  ]
                ),
              ),
            ]
        )

In [23]:
from torch.utils.data import DataLoader, Dataset,ConcatDataset,default_collate
train_dataset=labeled_video_dataset(val_df,
                   clip_sampler=make_clip_sampler(clipmode, video_duration),\
                    transform=video_transform, decode_audio=False
                                   )

train_loader=DataLoader(train_dataset,batch_size=4,
           num_workers=0,
           pin_memory=True)

In [28]:
from time import sleep
import torchvision.models as models
import timm
from pytorch_lightning import seed_everything, LightningModule, Trainer
class OurModel(LightningModule):
    def __init__(self):
        super(OurModel,self).__init__()
        self.scheduler=scheduler
        self.video_model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
        self.video_model.projection.model=nn.Linear(in_features=2048, out_features=1000, bias=True)
        self.relu=nn.ReLU()
        self.linear=nn.Linear(1000,1)
        self.lr=1e-3
        self.batch_size=batch_size
        self.numworker=2
        self.metric = torchmetrics.Accuracy(task="binary")
        self.criterion=nn.BCEWithLogitsLoss()

    def forward(self,video):
        x=self.video_model(video)
        x=self.relu(x)
        x=self.linear(x)
        return x

    def configure_optimizers(self):
        opt=torch.optim.AdamW(params=self.parameters(),lr=self.lr )
        if self.scheduler=='cosine':
            scheduler=CosineAnnealingLR(opt,T_max=10,  eta_min=1e-6, last_epoch=-1)
            return {'optimizer': opt,'lr_scheduler':scheduler}
        elif self.scheduler=='reduce':
            scheduler=ReduceLROnPlateau(opt,mode='min', factor=0.5, patience=5)
            return {'optimizer': opt,'lr_scheduler':scheduler,'monitor':'val_loss'}
        elif self.scheduler=='warm':
            scheduler=CosineAnnealingWarmRestarts(opt,T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1)
            return {'optimizer': opt,'lr_scheduler':scheduler}
        elif self.scheduler=='cycle':
            opt=torch.optim.AdamW(params=self.parameters(),lr=1e-6 )
            scheduler=OneCycleLR(opt,max_lr=1e-2,epochs=15,steps_per_epoch=len(self.train_df)//self.batch_size//4)
            lr_scheduler = {'scheduler': scheduler, 'interval': 'step'}
            return {'optimizer': opt, 'lr_scheduler': lr_scheduler}
        elif self.scheduler=='lambda':
            lambda1 = lambda epoch: 0.9 ** epoch
            scheduler = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda=lambda1)
            return {'optimizer': opt, 'lr_scheduler': scheduler}
        elif self.scheduler=='constant':
            return opt

    def train_dataloader(self):
        dataset=labeled_video_dataset(train_df,
                   clip_sampler=make_clip_sampler(clipmode, video_duration),\
                    transform=video_transform, decode_audio=False)

        loader=DataLoader(dataset,batch_size=self.batch_size,
                   num_workers=self.numworker,
                   pin_memory=True)
        return loader

    def training_step(self,batch,batch_idx):
        video,label=batch['video'],batch['label']
#         label=label.ravel().to(torch.int64)
        out = self(video)
        loss=self.criterion(out,label)
        metric=self.metric(out,label.to(torch.int64))
        return {'loss':loss,'metric':metric.detach()}

    def on_train_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().cpu().numpy().round(2)
        metric=torch.stack([x["metric"] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('train_loss', loss,batch_size=self.batch_size)
        self.log('train_metric', metric,batch_size=self.batch_size)
        print('training loss ',self.current_epoch,loss,metric)

    def val_dataloader(self):
        dataset=labeled_video_dataset(val_df,
                   clip_sampler=make_clip_sampler(clipmode, video_duration),\
                    transform=video_transform, decode_audio=False)

        loader=DataLoader(dataset,batch_size=self.batch_size,
                   num_workers=self.numworker,
                   pin_memory=True)
        return loader

    def validation_step(self,batch,batch_idx):
        video,label=batch['video'],batch['label']
        out = self(video)
        loss=self.criterion(out,label)
        metric=self.metric(out,label.to(torch.int64))
        return {'loss':loss,'metric':metric.detach()}

    def on_validation_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().cpu().numpy().round(2)
        metric=torch.stack([x["metric"] for x in outputs]).mean().cpu().numpy().round(2)
        print('validation loss ',self.current_epoch,loss,metric)
        self.log('val_loss', loss,batch_size=self.batch_size)
        self.log('val_metric',metric,batch_size=self.batch_size)

    def test_dataloader(self):
        dataset=labeled_video_dataset(val_df,
                   clip_sampler=make_clip_sampler(clipmode, video_duration),\
                    transform=video_transform, decode_audio=False)

        loader=DataLoader(dataset,batch_size=self.batch_size,
                   num_workers=self.numworker,
                   pin_memory=True)
        return loader

    def test_step(self, batch, batch_idx):
        video,label=batch['video'],batch['label']
        out = self(video)
        return { 'label': label.detach(), 'pred': out.detach()}

    def test_epoch_end(self, outputs):
        label = torch.cat([x['label'] for x in outputs]).cpu().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()
        pred=np.where(pred>0.5,1,0)
        print(classification_report(label, pred))

In [25]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss',dirpath='checkpoints',
                                        filename='file',save_last=True)
lr_monitor = LearningRateMonitor(logging_interval='epoch')

In [26]:
model=OurModel()
seed_everything(0)
trainer = Trainer(max_epochs=30,
#                 deterministic=True,
                accelerator='gpu', devices=-1,
                  precision=16,
                accumulate_grad_batches=2,
                enable_progress_bar = True,
                num_sanity_val_steps=0,
                  callbacks=[lr_monitor,checkpoint_callback])

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
INFO:lightning_fabric.utilities.seed:Global seed set to 0
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type              | Params
--------------------------------------------------
0 | video_model | EfficientX3d      | 5.0 M 
1 | relu        | ReLU              | 0     
2 | linear      | Linear            | 1.0 K 
3 | metric      | BinaryAccuracy    | 0     
4 | criterion   | BCEWithLogitsLoss | 0     
--------------------------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.0 M     Total params
20.099    Total estimated model params size (MB)
